In [8]:
# imported dependencies
import pennylane as qml
from itertools import product
import numpy as np
import numpy.random as random
import stabilizer as stabilizers
qml.version()

'0.29.1'

<img src="numerical_sim.png" alt="Drawing" style="height: 200px;"/>


In [17]:
def get_unitary_gate(ZL, XL):
  theta_i = random.uniform(0, 2*np.pi)
  theta_i_prime = random.uniform(0, 2*np.pi)
  return np.exp(1j * theta_i * ZL) * np.exp(1j * theta_i_prime * XL)

In [24]:
test = stabilizers.get_4_1_2()

XL = test[-1]
ZL = test[-2]

get_unitary_gate(ZL, XL)
print("ok")

ok
